In [1]:
import os
import sys
import pandas as pd
from sklearn.model_selection import train_test_split
sys.path.insert(0, os.path.abspath('../')) # needed to import src
from src.data_cleaning import WaseemTwitterPreparer, DavidsonTwitterPreparer, FountaTwitterPreparer, \
    GolbeckTwitterPreparer
OUTPUT_DIRECTORY = '../data'

In [2]:
davidson_preparer = DavidsonTwitterPreparer(verbose=True, path_to_raw='../data/twitter_datasets/davidson/davidson_labelled.csv')
davidson_preparer.load_data()
davidson_preparer.prepare_data()

11/01/2020 10:27:45 PM Uncleaned Twitter Davidson Shape: (24783, 6)


In [3]:

waseem_preparer = WaseemTwitterPreparer(verbose=True, path_to_raw=['../data/twitter_datasets/waseem/NAACL_SRW_2016.csv', '../data/twitter_datasets/waseem/NLP+CSS_2016.csv'])
waseem_preparer.load_data()
waseem_preparer.prepare_data()


C:\Users\matan\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3254: DtypeWarning: Columns (47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,

In [4]:
golbeck_preparer = GolbeckTwitterPreparer(verbose=True, path_to_raw='../data/twitter_datasets/golbeck/onlineHarassmentDataset.csv')
golbeck_preparer.load_data()
golbeck_preparer.prepare_data()

11/01/2020 10:27:49 PM Uncleaned Twitter Golbeck Shape: (20360, 2)


In [5]:
founta_preparer = FountaTwitterPreparer(verbose=True, path_to_raw='../data/twitter_datasets/founta/hatespeechtwitter.csv')
founta_preparer.load_data()
founta_preparer.prepare_data()

11/01/2020 10:27:49 PM Uncleaned Twitter Founta Shape: (80000, 2)


In [6]:
founta = founta_preparer.get_cleaned_data()
davidson = davidson_preparer.get_cleaned_data()
waseem = waseem_preparer.get_cleaned_data()
golbeck = golbeck_preparer.get_cleaned_data()

In [7]:
founta.loc[:,'original_ds'] = 'founta'
davidson.loc[:,'original_ds'] = 'davidson'
waseem.loc[:,'original_ds'] = 'waseem'
golbeck.loc[:,'original_ds'] = 'golbeck'

In [8]:
print('founta -  shape:', founta.shape, ' - label distribution:', round((founta[founta['is_hate']==1].shape[0] / founta.shape[0]), 3), '- columns:',founta.columns)
print('davidson -  shape:', davidson.shape, ' - label distribution:', round((davidson[davidson['is_hate']==1].shape[0] / davidson.shape[0]), 3), '- columns:', davidson.columns)
print('waseem -  shape:', waseem.shape, ' - label distribution:', round((waseem[waseem['is_hate']==1].shape[0] / waseem.shape[0]), 3), '- columns:', waseem.columns)
print('golbeck -  shape:', golbeck.shape, ' - label distribution:', round((golbeck[golbeck['is_hate']==1].shape[0] / golbeck.shape[0]), 3), '- columns:', golbeck.columns)
# Waiting for Founta to respond to my email as 30k tweets are missing from dataset, many more is_hate

founta -  shape: (50487, 11)  - label distribution: 0.037 - columns: Index(['tweet_id', 'maj_label', 'is_hate', 'is_abusive', 'id', 'text',
       'cleaned_tweet', 'dialect_prs', 'is_aae_08', 'is_aae_06',
       'original_ds'],
      dtype='object')
davidson -  shape: (24783, 13)  - label distribution: 0.058 - columns: Index(['count', 'hate_speech', 'offensive_language', 'neither', 'class',
       'tweet', 'cleaned_tweet', 'is_hate', 'is_offensive', 'dialect_prs',
       'is_aae_08', 'is_aae_06', 'original_ds'],
      dtype='object')
waseem -  shape: (16631, 12)  - label distribution: 0.204 - columns: Index(['tweet_id', 'label', 'is_racism', 'is_sexism', 'is_hate', 'id', 'text',
       'cleaned_tweet', 'dialect_prs', 'is_aae_08', 'is_aae_06',
       'original_ds'],
      dtype='object')
golbeck -  shape: (19718, 8)  - label distribution: 0.241 - columns: Index(['Code', 'Tweet', 'is_hate', 'cleaned_tweet', 'dialect_prs', 'is_aae_08',
       'is_aae_06', 'original_ds'],
      dtype='obje

In [9]:
founta.merge(waseem, how='inner', on='tweet_id') # no overlap on tweet_id

,tweet_id,maj_label,is_hate_x,is_abusive,id_x,text_x,cleaned_tweet_x,dialect_prs_x,is_aae_08_x,is_aae_06_x,...,is_racism,is_sexism,is_hate_y,id_y,text_y,cleaned_tweet_y,dialect_prs_y,is_aae_08_y,is_aae_06_y,original_ds_y


In [10]:
davidson.rename(columns={"tweet":"text"},inplace=True)
golbeck.rename(columns={"Tweet":"text"},inplace=True)
overlap_cols = list(set(founta.columns).intersection(set(davidson.columns), set(waseem.columns),set(golbeck.columns)))

In [11]:
merged_df = pd.concat([founta, davidson, waseem, golbeck])

In [12]:
merged_df = merged_df[overlap_cols]
merged_df.drop_duplicates(inplace=True, subset=['cleaned_tweet']) #7338 duplicate tweets
merged_df.to_csv('../data/twitter_datasets/cleaned/all_datasets_merged.csv',index=False)

In [13]:
merged_df

,is_aae_06,dialect_prs,original_ds,cleaned_tweet,is_hate,is_aae_08,text
0,0,[0.10533575 0.36970896 0.09850918 0.42644611],founta,Alex Brosas another idiot #ALDUBKSGoesToUS,0,0,Alex Brosas another idiot #ALDUBKSGoesToUS ht...
1,0,[0.16670355 0.37222626 0.12378758 0.3372826 ],founta,"RT : as Nancy Reagan would say, 'just say FUCK...",0,0,"RT @ItIzBiz: as Nancy Reagan would say, 'just ..."
2,0,[0.06410871 0.12914915 0.16314298 0.64359916],founta,RT : The Nazi death gas so horrific even Hitle...,0,0,RT @MailOnline: The Nazi death gas so horrific...
3,0,[0.16246387 0.31474683 0.01293354 0.50985576],founta,I hate er chase because if the Bitch that work...,1,0,I hate er chase because if the Bitch that work...
4,0,[0.56263761 0.19150091 0.09204273 0.15381875],founta,RT : Pineapples do not belong on pizza. Y'all ...,1,0,RT @nyctophil3: Pineapples do not belong on pi...
...,...,...,...,...,...,...,...
19713,0,[0.14449078 0.16982948 0.24125604 0.4444237 ],golbeck,Wise words by Ben Shapiro https: / /t.co /ldBj...,1,0,Wise words by Ben Shapiro https: / /t.co /ldBj...
19714,0,[0.05411577 0.10954312 0.01288798 0.82345312],golbeck,Women are much more likely to be liberals than...,1,0,Women are much more likely to be liberals than...
19715,0,[0.17427205 0.34565292 0.09695176 0.38312327],golbeck,Would you like me to release to you this King ...,1,0,Would you like me to release to you this King ...
19716,0,[0.09649005 0.69237505 0.00770617 0.20342874],golbeck,You Fucking Nigger You Did It To Yourself You ...,1,0,You Fucking Nigger You Did It To Yourself You ...


In [14]:
overlap_cols

['is_aae_06',
 'dialect_prs',
 'original_ds',
 'cleaned_tweet',
 'is_hate',
 'is_aae_08',
 'text']

In [76]:
merged_df['is_aae'].sum() #673 maybe too high of a threshold?

673

In [ ]:
# 11187/merged_df[merged_df['is_aae']==0].shape[0] # 0.10797428769979152 11187/103608

In [79]:
# 45/merged_df[merged_df['is_aae08']==1].shape[0] #0.06686478454680535 45/673

0.06686478454680535

In [15]:
def _create_test_train_dev_splits(clean_data, tr_size=0.8, dev_test_split=0.5, stratify_label='is_hate'):
    train, valid_test = train_test_split(clean_data, train_size=tr_size, random_state=42,
                                         stratify=clean_data[[stratify_label]])
    test, valid = train_test_split(valid_test, train_size=dev_test_split, random_state=42,
                                   stratify=valid_test[stratify_label])

    print(f"Train - # of is_hate: {train[train['is_hate'] == 1].shape[0]}, total instances: " +
                f"{train.shape[0]}, percent of positive: {train[train['is_hate'] == 1].shape[0] / train.shape[0]}")
    print(f"Test - # of is_hate: {test[test['is_hate'] == 1].shape[0]}, total instances: " +
                f"{test.shape[0]}, percent of positive: {test[test['is_hate'] == 1].shape[0] / test.shape[0]}")
    print(f"Test - # of is_hate: {valid[valid['is_hate'] == 1].shape[0]}, total instances: " +
                    f"{valid.shape[0]}, percent of positive: {valid[valid['is_hate'] == 1].shape[0] / valid.shape[0]}")

    return train, test, valid

def save_test_train_dev_splits(ds):
    twitter_train, twitter_valid, twitter_test = _create_test_train_dev_splits(ds, tr_size=0.8,
                                                                                    dev_test_split=0.5,
                                                                                    stratify_label='is_hate')
    # train: 861 7879 0.1093%   test: 214 1962 0.1091    dev: 121 1103 0.1097%

    twitter_train.to_csv('../data/twitter_datasets/combined/train.csv', index=False)
    twitter_valid.to_csv('../data/twitter_datasets/combined/dev.csv', index=False)
    twitter_test.to_csv('../data/twitter_datasets/combined/test.csv', index=False)

In [16]:

save_test_train_dev_splits(merged_df)

C:\Users\matan\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Train - # of is_hate: 8986, total instances: 83424, percent of positive: 0.10771480629075565
Test - # of is_hate: 1123, total instances: 10428, percent of positive: 0.10769083237437668
Test - # of is_hate: 1123, total instances: 10429, percent of positive: 0.10768050628056382


In [5]:

x = pd.read_csv('../data/nyt_corpus/nyt_data/nyt_keyword_sample/test.csv', index_col=0)

In [7]:
x.to_csv('../data/nyt_corpus/nyt_data/nyt_keyword_sample/test.tsv', sep='\t')

In [17]:
pd.read_json('../data/gab/majority_gab_dataset_25k/test.jsonl', lines=True)

,Text,text_id,is_hate,hd,cv,vo,rel,rae,sxo,gen,idl,nat,pol,mph,ex,im,dialect_prs,is_aae_08,is_aae_06,target_pop
0,Why would u listen to that Jew garbage?,18176,1.0,1.0,0,1.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,1.0,0.0,"[0.26591279100000004, 0.3453043918, 0.06949236...",0,0,rel
1,This is not the mark of the beast,25397,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,"[0.1353630198, 0.27797185150000003, 0.06072240...",0,0,no_tp
2,"I know the Marxists didn't coin the phrase, bu...",1367,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,"[0.061601486500000004, 0.1346841258, 0.1308077...",0,0,rae
3,What did Kanye West mean yesterday at the Whit...,21299,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,"[0.3103075684, 0.2632882146, 0.0625981568, 0.3...",0,0,no_tp
4,Ohhhhh well.... that be great. Nobody dead.......,7115,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,"[0.2836492217, 0.1745421607, 0.1805916226, 0.3...",0,0,no_tp
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1655,"Elise, we will never know for sure.",15123,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,"[0.24625772370000001, 0.3301669695, 0.00033036...",0,0,no_tp
1656,Paul Nehlen has turned into an absolute shitlo...,597,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,"[0.0679844678, 0.1022581068, 0.007578780100000...",0,0,no_tp
1657,Colt Mighty Mouth Masturntator Review: http:/...,24017,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,"[0.2379950102, 0.13391481530000002, 0.09466192...",0,0,no_tp
1658,"Aw. One of mine (I have 5 adults, too) loves t...",9685,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,"[0.2033540046, 0.36175997400000004, 0.01176380...",0,0,no_tp


In [20]:
pd.DataFrame({'a':[1,2],'b':[3,4]}).shape == (2,2)

True

In [25]:
import csv
pd.read_csv('../data/nyt_corpus/nyt_data/nyt_keyword_sample/test.tsv', delimiter='\t')

f = open('../data/nyt_corpus/nyt_data/nyt_keyword_sample/test.tsv')
reader = csv.reader(f, delimiter='\t')
next(reader) # skip header
examples = []
for i, row in enumerate(reader):
    print(row)

['2116647', "sopes, a mexican ''pizza,'' is a soft corn pancake topped with refried beans, green salsa, melted crumbled white cheese and your choice of meat.", '0', 'mexican']
['1043612', 'for many black south africans, the prospects of a better life remain dim.', '0', 'africans']
['449364', 'they sought a safe haven where they could continue to live according to jewish tradition and law.', '0', 'jewish']
['2322717', 'such black-and-white rules were well suited to the hand-to-hand or horse-to-horse combat characteristic of limited medieval wars.', '0', 'black']
['1242853', "if arabs and jews can peacefully co-exist in jerusalem's talpiot, they can peacefully co-exist on the mount of olives.", '0', 'jews']
['908357', 'mr. vacco himself seemed to suggest that ms. burstein\'s homosexuality was a proxy for her views on various social and criminal justice issues, and he issued a 60-second radio commercial yesterday calling her "the most liberal person ever to run for statewide office in new

In [ ]:
'''
stats
'''

In [2]:
all_twitter =pd.read_csv('../data/twitter_datasets/cleaned/all_datasets_merged.csv')

In [3]:
print(all_twitter[all_twitter['is_hate']==1].shape[0])
print(all_twitter[all_twitter['is_hate']==0].shape[0])

11232
93049
